In [1]:
from stellargraph import StellarGraph
import pandas as pd



2022-02-03 17:15:18.934042: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-03 17:15:19.312780: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-03 17:15:19.313395: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-03 17:15:19.350497: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

In [5]:
from shared.schema import DatasetSchema, GraphSchema
from shared.graph import DataGraph, igraph_to_nx
from shared.graph.loading import pd_from_entity_schema


In [6]:
DATASET = DatasetSchema.load_schema('star-wars')
schema = GraphSchema.from_dataset(DATASET)

In [38]:
explicit_label = False
explicit_timestamp = True
unix_timestamp = True
prefix_id = None
include_properties = []

nodes_dfs = {
    label: pd_from_entity_schema(
        entity_schema,
        explicit_label=explicit_label,
        explicit_timestamp=explicit_timestamp,
        include_properties=include_properties,
        unix_timestamp=unix_timestamp,
        prefix_id=prefix_id,
    ).set_index('id').drop(columns=['type'])
    for label, entity_schema in schema.nodes.items()
}

edges_dfs = {
    label: pd_from_entity_schema(
        entity_schema,
        explicit_label=explicit_label,
        explicit_timestamp=explicit_timestamp,
        include_properties=include_properties,
        unix_timestamp=unix_timestamp,
        prefix_id=prefix_id,
    ).reset_index().drop(columns=['type']).drop_duplicates(subset=['src', 'dst', 'timestamp'])
    for label, entity_schema in schema.edges.items()
}

cursor = 0
for df in edges_dfs.values():
    df.index += cursor
    cursor += len(df)

In [39]:
graph = StellarGraph(
    nodes=nodes_dfs,
    edges=edges_dfs,
    source_column='src',
    target_column='dst',
)
print(graph.info())

StellarGraph: Undirected multigraph
 Nodes: 113, Edges: 2078

 Node types:
  Character: [113]
    Features: none
    Edge types: Character-INTERACTIONS->Character, Character-MENTIONS->Character

 Edge types:
    Character-MENTIONS->Character: [1120]
        Weights: all 1 (default)
        Features: float32 vector, length 2
    Character-INTERACTIONS->Character: [958]
        Weights: all 1 (default)
        Features: float32 vector, length 2


In [40]:
batch_size = 200
epochs = 20
# Use 70% of edges for training, the rest for testing:
train_size = 0.7
test_size = 0.3

In [ ]:
from sklearn import model_selection

edges_train, edges_test = model_selection.train_test_split(
    edges_with_ratings, train_size=train_size, test_size=test_size
)

edgelist_train = list(edges_train[["user_id", "movie_id"]].itertuples(index=False))
edgelist_test = list(edges_test[["user_id", "movie_id"]].itertuples(index=False))

labels_train = edges_train["rating"]
labels_test = edges_test["rating"]

In [43]:
num_samples = [8, 4]

In [44]:
edges = graph.edges(use_ilocs=True)

In [45]:
from stellargraph.mapper import HinSAGELinkGenerator

generator = HinSAGELinkGenerator(
    graph, batch_size, num_samples, head_node_types=["Character", "Character"]
)

RuntimeError: This StellarGraph has no numeric feature attributes for nodesNode features are required for machine learning